In [1]:
!pip install corner

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd drive/My\ Drive/Projects/Autoencoder_1

/content/drive/My Drive/Projects/Autoencoder_1


In [0]:
import os
import utils
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import corner.corner as corner
import matplotlib.pyplot as plt

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [0]:
import fastai
from fastai import train as tr 
from fastai.callbacks import ActivationStats
from fastai import data_block, basic_train, basic_data

In [0]:
import matplotlib as mpl
import my_matplotlib_style as ms
mpl.rc_file('my_matplotlib_rcparams')

In [0]:
from utils import plot_activations
from my_utils import make_plots, load_data, train_evaluate_model

In [0]:
if torch.cuda.is_available():
  fastai.torch_core.defaults.device = 'cuda'

In [0]:
class AE_3D_200_relu(nn.Module):
    def __init__(self, hidden_dim_1, hidden_dim_2, hidden_dim_3, n_features=4):
        super(AE_3D_200_relu, self).__init__()
        self.en1 = nn.Linear(n_features, hidden_dim_1)
        self.en2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.en3 = nn.Linear(hidden_dim_2, hidden_dim_3)
        self.en4 = nn.Linear(hidden_dim_3, 3)
        self.de1 = nn.Linear(3, hidden_dim_3)
        self.de2 = nn.Linear(hidden_dim_3, hidden_dim_2)
        self.de3 = nn.Linear(hidden_dim_2, hidden_dim_1)
        self.de4 = nn.Linear(hidden_dim_1, n_features)
        self.relu = nn.ReLU()

    def encode(self, x):
        return self.en4(self.relu(self.en3(self.relu(self.en2(self.relu(self.en1(x)))))))

    def decode(self, x):
        return self.de4(self.relu(self.de3(self.relu(self.de2(self.relu(self.de1(self.relu(x))))))))

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)

    def describe(self):
        return 'in-' + str(hidden_dim_1) + '-' + str(hidden_dim_2) + '-' + str(hidden_dim_3) + '-' +'3-' + str(hidden_dim_3) + '-' + str(hidden_dim_2) + '-' + str(hidden_dim_1) + '-out'

In [0]:
class AE_3D_200_tanh(nn.Module):
    def __init__(self, hidden_dim_1, hidden_dim_2, hidden_dim_3, n_features=4):
        super(AE_3D_200_tanh, self).__init__()
        self.en1 = nn.Linear(n_features, hidden_dim_1)
        self.en2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.en3 = nn.Linear(hidden_dim_2, hidden_dim_3)
        self.en4 = nn.Linear(hidden_dim_3, 3)
        self.de1 = nn.Linear(3, hidden_dim_3)
        self.de2 = nn.Linear(hidden_dim_3, hidden_dim_2)
        self.de3 = nn.Linear(hidden_dim_2, hidden_dim_1)
        self.de4 = nn.Linear(hidden_dim_1, n_features)
        self.tanh = nn.Tanh()

    def encode(self, x):
        return self.en4(self.tanh(self.en3(self.tanh(self.en2(self.tanh(self.en1(x)))))))

    def decode(self, x):
        return self.de4(self.tanh(self.de3(self.tanh(self.de2(self.tanh(self.de1(self.tanh(x))))))))

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)

    def describe(self):
        return 'in-' + str(hidden_dim_1) + '-' + str(hidden_dim_2) + '-' + str(hidden_dim_3) + '-' +'3-' + str(hidden_dim_3) + '-' + str(hidden_dim_2) + '-' + str(hidden_dim_1) + '-out'

In [0]:
hidden_dim_1 = 200
hidden_dim_2 = 100
hidden_dim_3 = 50

In [13]:
model = AE_3D_200_tanh(hidden_dim_1, hidden_dim_2, hidden_dim_3)
model_name = 'AE_3D_200_tanh'
num_epochs = 50
learning_rate = 3e-4
loss = train_evaluate_model(model, model_name, num_epochs, learning_rate, hidden_dim_1, hidden_dim_2, hidden_dim_3)

epoch,train_loss,valid_loss,time
0,0.035177,0.035388,00:04
1,0.033667,0.032700,00:04
2,0.030812,0.032021,00:04
3,0.029041,0.030537,00:04
4,0.028227,0.029918,00:04
5,0.029323,0.029837,00:04
6,0.029705,0.029293,00:03
7,0.028958,0.028969,00:04
8,0.027423,0.027484,00:04
9,0.025306,0.025425,00:04


In [15]:
model = AE_3D_200_relu(hidden_dim_1, hidden_dim_2, hidden_dim_3)
model_name = 'AE_3D_200_relu'
num_epochs = 50
learning_rate = 3e-3
loss = train_evaluate_model(model, model_name, num_epochs, learning_rate, hidden_dim_1, hidden_dim_2, hidden_dim_3)

epoch,train_loss,valid_loss,time
0,0.080603,0.083568,00:04
1,0.070086,0.069681,00:03
2,0.061571,0.061208,00:04
3,0.061332,0.063485,00:04
4,0.057889,0.056934,00:04
5,0.056298,0.054582,00:04
6,0.053316,0.052067,00:03
7,0.064156,0.066051,00:04
8,0.063222,0.062717,00:04
9,0.054536,0.056537,00:03
